In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export utils

# Utils

> Utilities

In [ ]:
%nbdev_export
import numpy as np
from fastcore.all import *
from fastai.interpret import *
from fastai.data.all import *

In [ ]:
%nbdev_export
def df_slicer(df, w, s=1, padding=False, padding_value=0, return_as='ndarray'):
    "Transform a numeric dataframe `df` into slices (np arrays) of `w` \
    rows and the same number of columns than the original dataframe. The \
    distance between each slice is given by the stride `s`. If `padding` is \
    equals to True, the last slices which have less than `w` points are filled \
    with the value marked in the argument `padding_value`. Otherwise, those \
    slices are removed from the result."
    aux = [df.iloc[x:x+w] for x in range(0, len(df), s)]
    if padding:
        with_padding = [x.append(pd.DataFrame(
            np.full((w - len(x), len(df.columns)), padding_value),
            columns=df.columns.values)) if len(x) < w else x for x in aux]
    else:
        with_padding = [x for x in aux if len(x) == w]
    return np.rollaxis(np.dstack([x.values for x in with_padding]), -1)

### Allow `ClassificationInterpretation.top_losses` to filter by error type

In [ ]:
%nbdev_export
class ClassificationInterpretationAugmented(ClassificationInterpretation):
    def top_losses(self, k=None, largest=True, predicted=None, actual=None):
        "`k` largest(/smallest) losses and indexes, defaulting to all losses (sorted by `largest`)."
        if predicted is None and actual is None:
            # Default behaviour
            return self.losses.topk(ifnone(k, len(self.losses)), largest=largest)
        else:
            # Subset losses by the conditions given in predicted and actual arguments
            cond_preds = (self.decoded == self.vocab.o2i[predicted]) if predicted else tensor(True)
            cond_actuals = (self.targs == self.vocab.o2i[actual]) if actual else tensor(True)
            idxs = (cond_preds & cond_actuals).nonzero().squeeze()
            loss_subset = self.losses[idxs].topk(ifnone(k, len(idxs)), largest=largest)
            # The indices in loss_subset are relative to the object `idxs`. We have to
            # return the aboluste idxs with respect to the `self` object.
            # TODO: It's returning a pair instead of a topk object
            return (loss_subset.values, idxs[loss_subset.indices])


    def plot_top_losses(self, k, largest=True, predicted=None,
                        actual=None, **kwargs):
        losses,idx = self.top_losses(k, largest, predicted, actual)
        if not isinstance(self.inputs, tuple): self.inputs = (self.inputs,)
        if isinstance(self.inputs[0], torch.Tensor): inps = tuple(o[idx] for o in self.inputs)
        else: inps = self.dl.create_batch(self.dl.before_batch([tuple(o[i] for o in self.inputs) for i in idx]))
        b = inps + tuple(o[idx] for o in (self.targs if is_listy(self.targs) else (self.targs,)))
        x,y,its = self.dl._pre_show_batch(b, max_n=k)
        b_out = inps + tuple(o[idx] for o in (self.decoded if is_listy(self.decoded) else (self.decoded,)))
        x1,y1,outs = self.dl._pre_show_batch(b_out, max_n=k)
        if its is not None:
            plot_top_losses(x, y, its, outs.itemgot(slice(len(inps), None)), self.preds[idx], losses,  **kwargs)